# OpenAlex와 KCI 자료 통합 for STM

In [28]:
import pandas as pd
import glob
from tqdm.auto import tqdm
tqdm.pandas()
import re

In [29]:
kci_file = glob.glob('./kci_data/*.xls')

In [30]:
kci = pd.DataFrame()
for file in tqdm(kci_file):
    temp = pd.read_excel(file)
    kci = pd.concat([kci, temp], ignore_index=True)

  0%|          | 0/4 [00:00<?, ?it/s]

In [31]:
# kci.loc[~kci['DOI'].isnull(), 'DOI'] = 'https://doi.org/' + kci.loc[~kci['DOI'].isnull(), 'DOI']
# kci['DOI'] = kci['DOI'].str.strip()
kci['DOI']

0       10.25024/review.2023.26.2.001
1       10.25024/review.2023.26.2.002
2       10.25024/review.2023.26.2.003
3       10.25024/review.2023.26.2.004
4       10.25024/review.2023.26.2.007
                    ...              
2080                              NaN
2081                              NaN
2082                              NaN
2083                              NaN
2084                              NaN
Name: DOI, Length: 2085, dtype: object

In [32]:
# 'http'로 시작하지 않는 DOI에 'https://doi.org/' 추가 (NA 제외)
kci.loc[kci['DOI'].notna() & ~kci['DOI'].str.startswith('http').fillna(False), 'DOI'] = \
    'https://doi.org/' + kci.loc[kci['DOI'].notna() & ~kci['DOI'].str.startswith('http').fillna(False), 'DOI']

# 'http'로 시작하는 DOI를 'https'로 변경 (NA 제외)
kci.loc[kci['DOI'].notna() & kci['DOI'].str.startswith('http').fillna(False), 'DOI'] = \
    kci.loc[kci['DOI'].notna() & kci['DOI'].str.startswith('http').fillna(False), 'DOI'].str.replace('http://', 'https://', regex=False)


/tmp/ipykernel_3350480/787711427.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'https://doi.org/' + kci.loc[kci['DOI'].notna() & ~kci['DOI'].str.startswith('http').fillna(False), 'DOI']
/tmp/ipykernel_3350480/787711427.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  kci.loc[kci['DOI'].notna() & ~kci['DOI'].str.startswith('http').fillna(False), 'DOI'] = \
/tmp/ipykernel_3350480/787711427.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(cop

In [33]:
kci['DOI']

0       https://doi.org/10.25024/review.2023.26.2.001
1       https://doi.org/10.25024/review.2023.26.2.002
2       https://doi.org/10.25024/review.2023.26.2.003
3       https://doi.org/10.25024/review.2023.26.2.004
4       https://doi.org/10.25024/review.2023.26.2.007
                            ...                      
2080                                              NaN
2081                                              NaN
2082                                              NaN
2083                                              NaN
2084                                              NaN
Name: DOI, Length: 2085, dtype: object

In [34]:
kci['DOI'] = kci['DOI'].str.strip()

In [35]:
kci['origin'] = 'in'
kci['oa_kci'] = 'kci'

In [36]:
kci_range = kci.groupby('학술지명')['발행연도'].agg(['min', 'max'])
print(kci_range)


                                  min   max
학술지명                                       
Acta Koreana                     2007  2023
Korea Journal                    2002  2024
Seoul Journal of Korean Studies  1988  2023
The Review of Korean Studies     2002  2023


In [37]:
df = pd.read_pickle('./240402_KS_works.pkl')
df['oa_kci'] = 'oa'
len(df)

3379

In [38]:
# 텍스트 마이닝용 필요 columns
df_filtered = df[['id','doi','journal','origin','title','publication_year','abstract','oa_kci','cited_by_count']]

In [39]:
kci_filtered = kci[['논문 ID','DOI','학술지명','origin','논문명','발행연도','초록','oa_kci']]
kci_filtered.columns = ['id','doi','journal','origin','title','publication_year','abstract','oa_kci']
kci_filtered['cited_by_count'] = pd.NA

/tmp/ipykernel_3350480/3660691786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kci_filtered['cited_by_count'] = pd.NA


In [40]:
df_filtered.loc[df_filtered['doi'].isin(kci_filtered['doi']), 'abstract'] = kci_filtered.loc[kci_filtered['doi'].isin(df_filtered['doi']), 'abstract']

In [41]:
df_filtered = df_filtered.merge(kci_filtered.dropna(subset='doi')[['doi', 'abstract']], on='doi', how='left', suffixes=('', '_kci'))
df_filtered['abstract'] = df_filtered['abstract_kci'].fillna(df_filtered['abstract'])
df_filtered = df_filtered.drop('abstract_kci', axis=1)

In [42]:
df_filtered

,id,doi,journal,origin,title,publication_year,abstract,oa_kci,cited_by_count
0,https://openalex.org/W4384432764,https://doi.org/10.1353/seo.2023.a902140,Seoul Journal of Korean Studies,in,Felled Forests and Fallowed Fields: Establishi...,2023,Goryeo (918–1392) was a politically and commer...,oa,1
1,https://openalex.org/W4384432829,https://doi.org/10.1353/seo.2023.a902142,Seoul Journal of Korean Studies,in,The History of the Formation of Silhak in Mode...,2023,This article examines the historical trajector...,oa,1
2,https://openalex.org/W1578642520,https://doi.org/10.1353/seo.2015.0003,Seoul Journal of Korean Studies,in,Diplomatic War: Inter-Korean Relations in the ...,2015,The period of the 1970s was a turning point in...,oa,5
3,https://openalex.org/W1590043962,https://doi.org/10.1353/seo.2013.0023,Seoul Journal of Korean Studies,in,More Valuable than Gold: Korean Tungsten and t...,2013,A central component of the Japanese government...,oa,5
4,https://openalex.org/W210363985,https://doi.org/10.1353/seo.2014.0005,Seoul Journal of Korean Studies,in,Parhae in Historiography and Archaeology: Inte...,2014,The history of the kingdom of Parhae (Ch. Boha...,oa,5
...,...,...,...,...,...,...,...,...,...
3374,https://openalex.org/W2115655314,https://doi.org/10.1353/ks.1992.0016,Korean Studies,out,<i>The Bronze Age in Korea</i> (review),1992,128BOOK REVIEWS rather than philosophy. And if...,oa,0
3375,https://openalex.org/W2154921451,https://doi.org/10.1353/ks.1992.0014,Korean Studies,out,&lt;i&gt;Korea Old and New: A History&lt;/i&gt...,1992,"Book Reviews Korea Old and New: A History, by ...",oa,0
3376,https://openalex.org/W2162052910,https://doi.org/10.1353/ks.1992.0007,Korean Studies,out,&lt;i&gt;Alliance Under Tension: The Evolution...,1992,BOOK REVIEWS123 These volumes are more than ju...,oa,0
3377,https://openalex.org/W4243393701,https://doi.org/10.1353/ks.1993.0020,Korean Studies,out,Contributors,1993,NaN,oa,0


In [43]:
kj_2022_2023 = kci_filtered[(kci_filtered['journal'] == 'Korea Journal') & (kci_filtered['publication_year'] >= 2022) & (kci_filtered['publication_year'] <= 2023)]

In [44]:
rks_2021_2023 = kci_filtered[(kci_filtered['journal'] == 'The Review of Korean Studies') & (kci_filtered['publication_year'] >= 2021) & (kci_filtered['publication_year'] <= 2023)]

In [45]:
df_filtered = pd.concat([df_filtered, kj_2022_2023, rks_2021_2023], ignore_index=True)

In [46]:
df_filtered.groupby('journal')['publication_year'].agg(['min', 'max'])

,min,max
journal,,
Acta Koreana,2000,2023
European Journal of Korean Studies,2015,2023
Korea Journal,1961,2023
Korean Studies,1977,2023
Seoul Journal of Korean Studies,1988,2023
The Journal of Korean Studies,1969,2024
The Review of Korean Studies,1997,2023


In [47]:
df_filtered = df_filtered[(df_filtered['publication_year']>=2000)&(df_filtered['publication_year']<=2023)].reset_index(drop=True)
df_filtered

,id,doi,journal,origin,title,publication_year,abstract,oa_kci,cited_by_count
0,https://openalex.org/W4384432764,https://doi.org/10.1353/seo.2023.a902140,Seoul Journal of Korean Studies,in,Felled Forests and Fallowed Fields: Establishi...,2023,Goryeo (918–1392) was a politically and commer...,oa,1
1,https://openalex.org/W4384432829,https://doi.org/10.1353/seo.2023.a902142,Seoul Journal of Korean Studies,in,The History of the Formation of Silhak in Mode...,2023,This article examines the historical trajector...,oa,1
2,https://openalex.org/W1578642520,https://doi.org/10.1353/seo.2015.0003,Seoul Journal of Korean Studies,in,Diplomatic War: Inter-Korean Relations in the ...,2015,The period of the 1970s was a turning point in...,oa,5
3,https://openalex.org/W1590043962,https://doi.org/10.1353/seo.2013.0023,Seoul Journal of Korean Studies,in,More Valuable than Gold: Korean Tungsten and t...,2013,A central component of the Japanese government...,oa,5
4,https://openalex.org/W210363985,https://doi.org/10.1353/seo.2014.0005,Seoul Journal of Korean Studies,in,Parhae in Historiography and Archaeology: Inte...,2014,The history of the kingdom of Parhae (Ch. Boha...,oa,5
...,...,...,...,...,...,...,...,...,...
2892,ART002721077,https://doi.org/10.25024/review.2021.24.1.009,The Review of Korean Studies,in,The Haunted Maiden Deities on Jeju Island,2021,Jeju Island reveals exemplary openness to ghos...,kci,<NA>
2893,ART002721082,https://doi.org/10.25024/review.2021.24.1.010,The Review of Korean Studies,in,A Study on Trauma Aspects in Cho Se-Hee’s Nove...,2021,The aim of this paper is to study the problems...,kci,<NA>
2894,ART002721086,https://doi.org/10.25024/review.2021.24.1.012,The Review of Korean Studies,in,A Chinese Scholar’s View on the Costume of Gog...,2021,NaN,kci,<NA>
2895,ART002721085,https://doi.org/10.25024/review.2021.24.1.011,The Review of Korean Studies,in,Current Status of Korean Studies in China,2021,NaN,kci,<NA>


In [48]:
len(df_filtered[pd.isna(df_filtered['abstract'])])

690

In [49]:
df_filtered.loc[df_filtered['abstract'].isnull(), 'abstract'] = df_filtered.loc[df_filtered['abstract'].isnull(), 'doi'].map(kci_filtered.set_index('doi').reset_index()['abstract'])

In [50]:
# df_filtered.loc[df_filtered['abstract'].isnull(), 'abstract'] = df_filtered.loc[df_filtered['abstract'].isnull()].apply(lambda row: kci_filtered[kci_filtered['title'].str.lower() == row['title'].lower()]['abstract'].values[0] if not kci_filtered[kci_filtered['title'].str.lower() == row['title'].lower()].empty else row['abstract'], axis=1)
df_filtered.loc[df_filtered['abstract'].isnull(), 'abstract'] = df_filtered.loc[df_filtered['abstract'].isnull()].apply(
    lambda row: kci_filtered[kci_filtered['title'].str.lower().str.strip() == row['title'].lower().strip()]['abstract'].values[0]
    if not kci_filtered[kci_filtered['title'].str.lower().str.strip() == row['title'].lower().strip()].empty else row['abstract'], axis=1)

In [51]:
df_filtered.loc[df_filtered['abstract'] == '.', 'abstract'] = pd.NA

In [52]:
# book review는 초록이 없는게 맞음. 따라서 지워야함.
# abstract 열에서 Book Review로 시작하면 pd.NA 처리
df_filtered.loc[df_filtered['abstract'].str.contains(r'Book Review.*', regex=True, na=False), 'abstract'] = pd.NA
df_filtered.loc[df_filtered['abstract'].str.contains(r'Reviewed by.*', regex=True, na=False), 'abstract'] = pd.NA
df_filtered.loc[df_filtered['title'].str.contains(r'\[Book Review.*', regex=True, na=False), 'abstract'] = pd.NA
df_filtered.loc[df_filtered['title'].str.contains(r'\[book review.*', regex=True, na=False), 'abstract'] = pd.NA
df_filtered.loc[df_filtered['title'].str.contains(r'\[On This Topic.*', regex=True, na=False), 'abstract'] = pd.NA

In [53]:
df_filtered['abstract'] = df_filtered['abstract'].str.replace('^Abstract', '', regex=True)

In [54]:
len(df_filtered[pd.isna(df_filtered['abstract'])])

914

In [183]:
pd.crosstab(df_filtered['oa_kci'], df_filtered['abstract'].isna())

abstract,False,True
oa_kci,,
kci,109,57
oa,1873,858


In [55]:
pd.crosstab(df_filtered['oa_kci'], df_filtered['abstract'].isna())


abstract,False,True
oa_kci,,
kci,109,57
oa,1874,857


In [56]:
df_filtered.to_pickle('./240407_KS_KCI_merged.pkl')